# Advanced decorators

## Plan

1. Decorator recap.
2. Decorators with arguments.
3. Classes as decorators.
4. Decorating classes.

## Reference material: https://decorators.mathspp.com/intro.html

In [1]:
import functools

def decorator(function_to_decorate):  # 1
    @functools.wraps(function_to_decorate)  # 3
    # 2                # 4
    def inner_function(*args, **kwargs):
        ...  # 5
        result = function_to_decorate(*args, **kwargs)  # 6
        ...  # 7
        return result  # 8

    return inner_function  # 9

In [5]:
def decorator(f):
    def wrapper(*args, **kwargs):
        ...
        result = f(*args, **kwargs)
        ...
        return result

    return wrapper

@decorator
def foo():
    pass

# same as

def foo():
    """Docstring."""
    pass

foo = decorator(foo)

In [4]:
?foo

Signature: foo(*args, **kwargs)
Docstring: <no docstring>
File:      /var/folders/29/cpfnqrmx0ll8m1vp9f9fmnx00000gn/T/ipykernel_10836/2138163077.py
Type:      function

In [9]:
from functools import wraps

def decorator(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        ...
        result = f(*args, **kwargs)
        ...
        return result

    return wrapper

@decorator
def foo():
    """Docstring"""
    pass

?foo

Signature: foo()
Docstring: Docstring
File:      /var/folders/29/cpfnqrmx0ll8m1vp9f9fmnx00000gn/T/ipykernel_10836/1764871718.py
Type:      function

In [13]:
import time

def f():
    time.sleep(1)
    return 2

In [12]:
f.__closure

2